In [2]:
# import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
# install new libraries
!pip install -q tensorflow_hub
!pip install -q tfds-nightly

     |████████████████████████████████| 3.4MB 9.5MB/s 


In [4]:
# import installed libraries
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [5]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is available


In [9]:
# download IMDB movie reviews dataset
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split = ('train[:60%]', 'train[60%:]', 'test'),
    as_supervised = True
)

In [11]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [12]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [13]:

# pre-trained embedding model from tensorflow hub
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding,
                           input_shape=[],
                           dtype=tf.string,
                           trainable=True
                           )

In [24]:
len(hub_layer(train_examples_batch[:3])[2])

20

In [33]:
# build model
model = tf.keras.models.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [34]:
# model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                672       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 401,237
Trainable params: 401,237
Non-trainable params: 0
_________________________________________________________________


In [35]:
# compile the trained model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy']
              )

In [36]:
# fit model
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=2)

Epoch 1/20
30/30 - 2s - loss: 0.7684 - accuracy: 0.5223 - val_loss: 0.6315 - val_accuracy: 0.5832
Epoch 2/20
30/30 - 2s - loss: 0.5636 - accuracy: 0.6843 - val_loss: 0.5238 - val_accuracy: 0.7229
Epoch 3/20
30/30 - 2s - loss: 0.4374 - accuracy: 0.7850 - val_loss: 0.4085 - val_accuracy: 0.8057
Epoch 4/20
30/30 - 2s - loss: 0.2869 - accuracy: 0.8895 - val_loss: 0.3336 - val_accuracy: 0.8559
Epoch 5/20
30/30 - 2s - loss: 0.2001 - accuracy: 0.9285 - val_loss: 0.3083 - val_accuracy: 0.8655
Epoch 6/20
30/30 - 2s - loss: 0.1565 - accuracy: 0.9469 - val_loss: 0.3105 - val_accuracy: 0.8703
Epoch 7/20
30/30 - 2s - loss: 0.1305 - accuracy: 0.9571 - val_loss: 0.3143 - val_accuracy: 0.8694
Epoch 8/20
30/30 - 2s - loss: 0.1137 - accuracy: 0.9637 - val_loss: 0.3280 - val_accuracy: 0.8720
Epoch 9/20
30/30 - 2s - loss: 0.1003 - accuracy: 0.9685 - val_loss: 0.3368 - val_accuracy: 0.8672
Epoch 10/20
30/30 - 2s - loss: 0.0909 - accuracy: 0.9716 - val_loss: 0.3509 - val_accuracy: 0.8718
Epoch 11/20
30/30 -

In [37]:
# model evaluation
results = model.evaluate(test_data.batch(512), verbose=2)

49/49 - 2s - loss: 0.5561 - accuracy: 0.8466


In [38]:
results

[0.5561081767082214, 0.8466399908065796]